# 코드 작성 스타일
- 텐서플로로 코드를 작성하는 스타일 습득

In [ ]:
# 1.x 설치 : 매직코드로 1.x를 사용한다고 지시
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
# 모듈 가져오기(Colab이 아니면 설치부터 해야됨)
import tensorflow as tf

In [ ]:
# 버전 확인(딥러닝할 때는 반드시 버전 확인부터 하고 들어가야 됨)
tf.__version__
# 기본 코랩 버전 : 2.3.0
# 버전 2.x 이상은 2021년 정도에 전환될 것으로 예정(전반적인 부분에서)
# 현재는 대부분의 코드가 1.x를 사용하고 있으므로 1.x로 진행
# 차후 버전 업을 진행

'1.15.2'

# 기본 스타일

- 2개 영역으로 구분하여 작성
- part1
  - 데이터가 흘러가는 플로우를 구성
  - 연산의 입력, 출력의 흐름을 설계(인공신경망 구성)
  - 학습 전에 최적화, 평가 등 관련 플로우 구성
  - 파이썬으로 작성
- part2
  - 실제 데이터를 주입하여 학습 수행
  - 세션이 처리, 실제는 C++이 일을 한다.
  - Python → Cython → C++과 연결되어 있음
    - I/O → with문
      - 세션 오픈
      - 데이터 주입
      - C++ 연산 수행 → GPU로 작업 가능
      - 연산이 끝나면 Python에게 전달
      - 결과 확인
      - 세션 클로즈
     

# 기본 형태 작성

In [ ]:
# 1. 데이터가 흘러가는 플로우 구성
# 상수
hi = tf.constant('Hi Tensorflow')
hi

<tf.Tensor 'Const:0' shape=() dtype=string>

In [ ]:
# 2. 세션을 열고 데이터를 주입, 연산 수행, 결과 돌려받아서 출력(여기서는 with 안 써봄)
sess = tf.Session()
# 데이터를 주입하고 연산 수행
tmp = sess.run(hi)
# 결과 출력
print(tmp)
# 세션 닫기
sess.close()

b'Hi Tensorflow'


In [ ]:
# 간결하게 표현
with tf.Session() as sess :
  # 실제 연산 수행은 C++이 고속연산을 통해서 처리했다.
  tmp = sess.run(hi)
  print(tmp)

b'Hi Tensorflow'


# 연산에 필요한 요소
- 연산에 필요한 요소 정의, 계산

In [ ]:
# 간단한 계산
# 상수
a = tf.constant(123)
b = tf.constant(500)
a, b
# Tensor가 딥러닝 시 연산의 작업 단위다. (n-dimension)
# 'Const_1:0' : 텐서의 이름
# shpae :  텐서의 차원

(<tf.Tensor 'Const_1:0' shape=() dtype=int32>,
 <tf.Tensor 'Const_2:0' shape=() dtype=int32>)

In [ ]:
# 계산을 정의
add_opr = a + b
add_opr

<tf.Tensor 'add_2:0' shape=() dtype=int32>

In [ ]:
# 실행
with tf.Session() as sess : 
  # 더하기라는 연산도 C++이 진행한다.
  # 세션을 열어서 데이터, 연산행위를 전달하여 결과를 받는다.
  print(sess.run(add_opr))

623


# 텐서플로의 텐서
- 텐서에 대한 정의

<img src="https://drive.google.com/uc?id=1dbfjdD-Os0And_28pvdmByP443zVe3wi" width="90%">

- 텐서플로의 연산의 데이터의 기본 단위 행렬 -> 텐서 -> 차원을 가지고 있다. 
- 행렬은 오직 수치만 가진다. -> 데이터는 무조건 수치로 표현되어야 한다.

<img src="https://drive.google.com/uc?id=19J0AEae-jVHX_7ig3l3dItO6n1oe0zb5" width="90%">

# 상수, 변수, 플레이스 홀더, 데이터 플로우 그래프

- 상수 : Constant
- 변수 : Variable
- 플레이스 홀더 : Placeholder
  - 데이터를 주입할 때 받아주는 그릇
- 데이터 플로우 그래프 : Data Flow Graph
  - 데이터가 흘러가는 관계를 표현한 것
  - 데이터 간의 관계, 입력과 출력에 대한 관계 -> 그래프

In [ ]:
# 상수
# 이름은 관계를 표현할 때 내부적으로 사용 -> 나중에 tensorboard를 통해서 관계도가 시각화될 때 확인 가능
a = tf.constant(100, name='a')
b = tf.constant(200, name='b')
c = tf.constant(300, name='c')
a, b, c 

(<tf.Tensor 'a:0' shape=() dtype=int32>,
 <tf.Tensor 'b:0' shape=() dtype=int32>,
 <tf.Tensor 'c:0' shape=() dtype=int32>)

In [ ]:
# 변수 : 변하는 값을 가지는 공간
v = tf.Variable(0, name='v')
v

<tf.Variable 'v:0' shape=() dtype=int32_ref>

In [ ]:
# 연산식, 계산식
calc_opr = a + b + c
calc_opr

<tf.Tensor 'add_4:0' shape=() dtype=int32>

In [ ]:
# 데이터 플로우 그래프
# 상수 3개를 더한 값(연산 결과)을 변수에 넣어라(assign해라)(데이터 플로우)
assign_add = tf.assign(v, calc_opr)
assign_add

<tf.Tensor 'Assign:0' shape=() dtype=int32_ref>

In [ ]:
# 실행
with tf.Session() as sess :
  res = sess.run(assign_add)
  print(res)

600


# 데이터 주입
- 플레이스 홀더(외부로부터 데이터가 주입될 때 받아주는 그릇)를 이용 
- 딥러닝에서 학습 수행 시 데이터를 넣어서 학습하는 행위와 동일한 형태
- 주입되는 데이터의 형태와 동일하게 플레이스 홀더의 shape을 만들어 주는 게 관건
  - 플레이스 홀더는 고정 크기 / 가변 크기, 2가지 형태로 존재한다.
  

## 고정 크기


In [ ]:
# 정수를 3개 입력받을 수 있다.
# shape은 리스트나 튜플 형태 가능
a = tf.placeholder(tf.int32, [3])
a

<tf.Tensor 'Placeholder:0' shape=(3,) dtype=int32>

In [ ]:
# 상수
b = tf.constant(2)
b

<tf.Tensor 'Const_3:0' shape=() dtype=int32>

In [ ]:
# 데이터 플로우 그래프
# a는 벡터, b는 스칼라 -> 벡터 * 스칼라
x_opr = a * b
x_opr

<tf.Tensor 'mul:0' shape=(3,) dtype=int32>

In [ ]:
# 연산 수행
with tf.Session() as sess :
  # feed_dict : 데이터를 주입할 때 사용하는 파라미터
  # 변수명이 key값, 주입할 데이터가 값(value)
  res = sess.run(x_opr, feed_dict={ a:[1,2,3] })
  print(res)

[2 4 6]


## 가변 크기
- 데이터가 몇 개인지 모르겠으나,
  - None으로 크기 부여

In [ ]:
a     = tf.placeholder(tf.int32, [None])
b     = tf.constant(2)
x_opr = a * b
with tf.Session() as sess :
  res = sess.run(x_opr, feed_dict={ a:[1,2,3] })
  print(res)

[2 4 6]


In [ ]:
with tf.Session() as sess :
  res = sess.run(x_opr, feed_dict={ a:[1,2,3,4,5] })
  print(res)

[ 2  4  6  8 10]


# Define by Run
- Define by Run 방식으로 처리

In [ ]:
# tensorflow 1.x 버전 기준
sess = tf.InteractiveSession()

In [ ]:
m1 = tf.constant( [ [1.,2.], [3.,4.] ] )
m1
# 2D-Tensor 혹은 2-Dimension 혹은 matrix

<tf.Tensor 'Const_7:0' shape=(2, 2) dtype=float32>

In [ ]:
m2 = tf.constant( [[100.], [200.]] )
m2

<tf.Tensor 'Const_8:0' shape=(2, 1) dtype=float32>

In [ ]:
# 행렬의 곱
tf.matmul(m1, m2).eval()

array([[ 500.],
       [1100.]], dtype=float32)